# NBA 2024-25: Slump Shots & Recovery Shots
## Notebook 01: Data Acquisition
This notebook handles the acquisition of raw, shot-level NBA data for the 2024-25 regular season (via the NBA API). Detailed shot-by-shot and shot location data is pulled for all players, creating a complete dataset that will serve as the foundation for all downstream analysis.

In [2]:
# Install NBA API
!pip install nba_api

In [3]:
# Import libraries
import pandas as pd
import time
from tqdm import tqdm
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import teams

In [4]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 160)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.2f}")

---
## Acquire Shot-Level Data
#### (for all 30 teams)

In [7]:
# Get team IDs
nba_teams = teams.get_teams()
team_ids = [team["id"] for team in nba_teams]

print(f"Number of teams: {len(team_ids)}")

Number of teams: 30


In [8]:
# --- Loop through all 30 teams and pull all shot data from the 2024-25 regular season ---
all_shots = []

for team_id in tqdm(team_ids, desc="Downloading shots"):

    shot_data = shotchartdetail.ShotChartDetail(
        team_id=team_id,
        player_id=0,   # 0 = all players on that team
        season_nullable="2024-25",
        season_type_all_star="Regular Season",
        context_measure_simple="FGA"
    )

    df = shot_data.get_data_frames()[0]
    all_shots.append(df)
    time.sleep(1.5)

In [9]:
# Combine into one DataFrame
shots_2024_25 = pd.concat(all_shots, ignore_index=True)

print(f"{shots_2024_25.shape[0]:,} rows")

219,527 rows


---
## Save

In [11]:
# Inspect structure
shots_2024_25.head(10)

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022400001,7,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,11,43,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,26,-168,205,1,0,20241112,BOS,ATL
1,Shot Chart Detail,0022400001,10,1630552,Jalen Johnson,1610612737,Atlanta Hawks,1,11,38,Missed Shot,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,13,-136,-1,1,0,20241112,BOS,ATL
2,Shot Chart Detail,0022400001,21,1630552,Jalen Johnson,1610612737,Atlanta Hawks,1,10,50,Made Shot,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,25,157,203,1,1,20241112,BOS,ATL
3,Shot Chart Detail,0022400001,34,1630811,Keaton Wallace,1610612737,Atlanta Hawks,1,9,47,Missed Shot,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-176,184,1,0,20241112,BOS,ATL
4,Shot Chart Detail,0022400001,36,203991,Clint Capela,1610612737,Atlanta Hawks,1,9,44,Made Shot,Putback Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,-25,8,1,1,20241112,BOS,ATL
5,Shot Chart Detail,0022400001,39,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,9,23,Made Shot,Cutting Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,10,13,1,1,20241112,BOS,ATL
6,Shot Chart Detail,0022400001,46,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,8,50,Missed Shot,Floating Jump shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,12,53,111,1,0,20241112,BOS,ATL
7,Shot Chart Detail,0022400001,48,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,8,47,Missed Shot,Putback Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-10,17,1,0,20241112,BOS,ATL
8,Shot Chart Detail,0022400001,50,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,8,46,Missed Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-10,1,1,0,20241112,BOS,ATL
9,Shot Chart Detail,0022400001,52,1642258,Zaccharie Risacher,1610612737,Atlanta Hawks,1,8,43,Missed Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,1,0,20241112,BOS,ATL


In [12]:
# Inspect columns
shots_2024_25.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING',
       'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM'],
      dtype='object')

In [13]:
# Save to parquet
shots_2024_25.to_parquet("nba_2024_25_shot_level_data_raw.parquet")